In [15]:
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd 
from urllib.parse import quote
import unicodedata, re
from tqdm.notebook import tqdm
import string
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

In [10]:
pwd

'/home/miwoos/WorkSpace/Next_Ez_jejudo'

In [46]:
insta_url = 'https://www.instagram.com'
# driver = webdriver.Chrome('D:\WorkSpace\Jeju_Electric_Vehicle_analysis\chromedriver.exe')  # 윈도우 ## open driver for window version
driver = webdriver.Chrome(ChromeDriverManager().install()) ## 리눅스 # 리눅스 ## open driver for linunx version
driver.get(insta_url)
time.sleep(1)
# with open('D:\WorkSpace\mykey\instar_pw.txt') as f:
#     password = f.read()
# with open('D:\WorkSpace\mykey\instar_id.txt') as f:
#     id = f.read()
id = '01056537972'
password='931811ji!'
input_id = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_id.clear()
input_id.send_keys(id)
time.sleep(2)
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(3)
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/home/miwoos/.wdm/drivers/chromedriver/linux64/99.0.4844.51/chromedriver] found in cache
/tmp/ipykernel_517750/4201083712.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) ## 리눅스 # 리눅스 ## open driver for linunx version
/tmp/ipykernel_517750/4201083712.py:12: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  input_id = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
/tmp/ipykernel_517750/4201083712.py:16: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  input_pwd = driver.find_element_by_name('password')
/tmp/ipykernel_517750/4201083712.py:22: DeprecationWarning: find_element_by_

In [19]:
df_name= pd.read_excel("D:\GoogleDrive\Data\Data\넥스트이지_골프장\제주골프장_정보(강지희).xlsx")

In [18]:
df_name= pd.read_excel("/home/miwoos/WorkSpace/GoogleDrive/넥스트이지!!/제주골프장_정보(강지희).xlsx")

In [19]:
golf_course_name=list(df_name.골프장명)
golf_course_name.append('제주CC')
type(golf_course_name)

list

In [20]:
keyword = golf_course_name[0]
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EA%B7%B8%EB%A6%B0%ED%95%84%EB%93%9CCC'

In [21]:
golf_course_name[0]

'그린필드CC'

In [22]:
driver.get(url)
time.sleep(7)

In [23]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

/tmp/ipykernel_517750/2042192687.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector('div._9AhH0').click()


In [41]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [42]:
soup

<html class="js logged-in client-root js-focus-visible sDN5V" lang="ko" style="--desktop-nav-height:60px; --in-feed-story-item-padding:14px;"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>#그린필드cc 해시태그 • Instagram 사진 및 동영상</title>
<meta content="noimageindex, noarchive" name="robots"/>
<meta content="default" name="apple-mobile-web-app-status-bar-style"/>
<meta content="yes" name="mobile-web-app-capable"/>
<meta content="#ffffff" name="theme-color"/>
<meta content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1, viewport-fit=cover" id="viewport" name="viewport"/>
<link href="/data/manifest.json" rel="manifest"/>
<link as="style" crossorigin="anonymous" href="/static/bundles/es6/ConsumerUICommons.css/13adfed49916.css" rel="preload" type="text/css"/>
<link as="style" crossorigin="anonymous" href="/static/bundles/es6/Consumer.css/d49271e93cef.css" rel="preload" type="text/css"/>
<link as="style" crossorigin="anonymous" h

In [26]:
# tags = re.findall(r'#[^\s#,\\]+', content)
# tags

NameError: name 'content' is not defined

In [51]:
date = soup.select_one('._1o9PC')['datetime'][:10]
date

'2022-03-21'

In [28]:
try:
    like = soup.select_one('.EDfFK.ygqzn').text[:]
except:
    like = 0
like

'1st_won_yi님 여러 명이 좋아합니다'

In [29]:
row = [content, date, like, tags]
row

NameError: name 'content' is not defined

In [30]:
# 첫번째 게시글에서 다음글로 이동
driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div/button/div').click()
time.sleep(2)

/tmp/ipykernel_517750/1498655362.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div/button/div').click()


In [31]:
# 두번째 게시글부터 다음글로 이동
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button').click()
time.sleep(2)

/tmp/ipykernel_517750/3315607414.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button').click()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[1]/div/div/div[2]/button"}
  (Session info: chrome=99.0.4844.82)
Stacktrace:
#0 0x55c17eedc7d3 <unknown>
#1 0x55c17ec38688 <unknown>
#2 0x55c17ec6ec21 <unknown>
#3 0x55c17ec6ede1 <unknown>
#4 0x55c17eca1d74 <unknown>
#5 0x55c17ec8c6dd <unknown>
#6 0x55c17ec9fa0c <unknown>
#7 0x55c17ec8c5a3 <unknown>
#8 0x55c17ec61ddc <unknown>
#9 0x55c17ec62de5 <unknown>
#10 0x55c17ef0d49d <unknown>
#11 0x55c17ef2660c <unknown>
#12 0x55c17ef0f205 <unknown>
#13 0x55c17ef26ee5 <unknown>
#14 0x55c17ef03070 <unknown>
#15 0x55c17ef42488 <unknown>
#16 0x55c17ef4260c <unknown>
#17 0x55c17ef5bc6d <unknown>
#18 0x7fe98fe0f609 <unknown>


In [32]:
# 5를 하기 위해 popup 화면을 없앰(즉, X 를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

/tmp/ipykernel_517750/1010647869.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/button"}
  (Session info: chrome=99.0.4844.82)
Stacktrace:
#0 0x55c17eedc7d3 <unknown>
#1 0x55c17ec38688 <unknown>
#2 0x55c17ec6ec21 <unknown>
#3 0x55c17ec6ede1 <unknown>
#4 0x55c17eca1d74 <unknown>
#5 0x55c17ec8c6dd <unknown>
#6 0x55c17ec9fa0c <unknown>
#7 0x55c17ec8c5a3 <unknown>
#8 0x55c17ec61ddc <unknown>
#9 0x55c17ec62de5 <unknown>
#10 0x55c17ef0d49d <unknown>
#11 0x55c17ef2660c <unknown>
#12 0x55c17ef0f205 <unknown>
#13 0x55c17ef26ee5 <unknown>
#14 0x55c17ef03070 <unknown>
#15 0x55c17ef42488 <unknown>
#16 0x55c17ef4260c <unknown>
#17 0x55c17ef5bc6d <unknown>
#18 0x7fe98fe0f609 <unknown>


In [52]:
def get_content(driver,soup):
    try:
        content = soup.select_one('.C4VMK').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = '내용이없습니다'    
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('._1o9PC')['datetime'][:10]
    try:
        like = soup.select_one('.EDfFK.ygqzn').text[:]
    except:
        like = 0
    return [content, date, like, tags]

In [53]:
게시글수=[]
results =[]
for i,keyword in enumerate(golf_course_name):
    search_url = 'https://www.instagram.com/explore/tags/'
    url = f'{search_url}{quote(keyword)}'
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.get(url)
    time.sleep(9)
    # try:
    게시글수.append(int(driver.find_element_by_xpath('/html/body/div[1]/section/main/header/div[2]/div/div[2]/div/span').text))
    driver.find_element_by_xpath('/html/body/div[1]/section/main/article/div[1]/div/div/div[1]/div[1]/a/div[1]/div[2]').click()
    time.sleep(10)
    for j in tqdm(range(게시글수[i])):
        if j == 0:
            row = get_content(driver,soup)
            row.append(keyword)
            results.append(row)
            driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div/button').click()
            time.sleep(4)
        try:
            row = get_content(driver,soup)
            row.append(keyword)
            results.append(row)
            driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div[2]/button').click()
            time.sleep(4)
        except:
            pass
    # except:
    #     results.append(['내용이없습니다','내용이없습니다','내용이없습니다','내용이없습니다','내용이없습니다',])

/tmp/ipykernel_517750/1422574.py:11: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  게시글수.append(int(driver.find_element_by_xpath('/html/body/div[1]/section/main/header/div[2]/div/div[2]/div/span').text))
/tmp/ipykernel_517750/1422574.py:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/div[1]/section/main/article/div[1]/div/div/div[1]/div[1]/a/div[1]/div[2]').click()


  0%|          | 0/961 [00:00<?, ?it/s]

TypeError: 'NoneType' object is not subscriptable

In [44]:
driver.quit()

In [18]:
results

[['jeju_kingkong_golf#골프 #골린이 #골린이성장일기⛳️ #구력6개월차 #언제싱글처보노#드라이버 #드라이버스윙 #제골조 #제주골프 #그린필드cc #비거리280m40주',
  '2021-06-07',
  '좋아요 47개',
  ['#골프',
   '#골린이',
   '#골린이성장일기⛳️',
   '#구력6개월차',
   '#언제싱글처보노',
   '#드라이버',
   '#드라이버스윙',
   '#제골조',
   '#제주골프',
   '#그린필드cc',
   '#비거리280m40주']],
 '그린필드CC',
 ['sxx_ahㅈHㅁI없는 골푸 🏌🏻\u200d♀️🥊늘 골프가 1순위였능데 이젠 ,,,, 안룡 ~#골스타그램 #그린필드cc #스윙 #티샷수정됨\xa0·\xa038주',
  '2021-06-22',
  '1st_won_yi님 여러 명이 좋아합니다',
  ['#골스타그램', '#그린필드cc', '#스윙', '#티샷수정됨']],
 '그린필드CC',
 ['y_s_w99오늘 바람 아예없고 그늘도 없고 진짜진짜 더운날 지금 현재 상태 살짝  더위먹은듯..어질어질하네예- 그린필드cc 어려운데 재밋뜨아 #제주골프 #그린필드cc #조인골프 #골프 #필드 #라운딩 #그린이어려운골프장 #오늘도 #굿샷32주',
  '2021-08-07',
  '좋아요 70개',
  ['#제주골프',
   '#그린필드cc',
   '#조인골프',
   '#골프',
   '#필드',
   '#라운딩',
   '#그린이어려운골프장',
   '#오늘도',
   '#굿샷32주']],
 '그린필드CC',
 ['jawoniing3박4일 제주 전지훈련 끝🙋🏻\u200d♀️⛳️수정됨\xa0·\xa010주',
  '2022-01-05',
  '좋아요 145개',
  []],
 '그린필드CC',
 ['jeju_kingkong_golf#골린이성장일기⛳️ #라베 #싱글 #골프 #그린필드cc #구력7개월차 #굿샷⛳️ #제주골퍼 #제골조밴드32주',
  '2021-08-04',
  '좋아요 44개',
  

In [35]:
df = pd.DataFrame(results, columns=['content','date','like','tags','골프장검색키워드'])
df.head()

,content,date,like,tags,골프장검색키워드
0,"sxx_ahㅈHㅁI없는 골푸 🏌🏻‍♀️🥊늘 골프가 1순위였능데 이젠 ,,,, 안룡 ...",2021-06-22,1st_won_yi님 여러 명이 좋아합니다,"[#골스타그램, #그린필드cc, #스윙, #티샷수정됨]",그린필드CC
1,y_s_w99오늘 바람 아예없고 그늘도 없고 진짜진짜 더운날 지금 현재 상태 살짝 ...,2021-08-07,좋아요 70개,"[#제주골프, #그린필드cc, #조인골프, #골프, #필드, #라운딩, #그린이어려...",그린필드CC
2,jawoniing3박4일 제주 전지훈련 끝🙋🏻‍♀️⛳️수정됨 · 10주,2022-01-05,좋아요 145개,[],그린필드CC
3,eunjung3070#5월25일#쩡원장은제주도✈️..#그린필드cc..제주는 바람이라...,2021-05-25,좋아요 405개,"[#5월25일, #쩡원장은제주도✈️.., #그린필드cc..제주는]",그린필드CC
4,thomas_kimjungwoo동생들 배려에 무서운 내기조에서 빠져서 제수씨들과 한...,2021-10-04,좋아요 46개,"[#골프그램⛳️, #제주도, #제주여행, #그린필드cc, #제주골프, #제골조, #...",그린필드CC


In [38]:
df.골프장검색키워드.value_counts()

그린필드CC     161
내용이없습니다     29
Name: 골프장검색키워드, dtype: int64